# Este es el Jupyter Notebook que se ha usado para hacer las prediciones de los models elegidos en el model_testing 

# Tambien se han aplicado aquí busqueda de modelos AutoML con H2o

In [ ]:
# %pip install h2o

In [ ]:
# %pip install auto-sklearn

In [ ]:
# %pip install xgboost

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor as GBR, AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from catboost import CatBoostRegressor as CBR

In [2]:
from sklearn.model_selection import GridSearchCV  
import xgboost as xgb
from xgboost import XGBRegressor
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from lightgbm import LGBMRegressor as LGBMR
import h2o


from h2o.automl import H2OAutoML

# Importación de los archivos "límpios"

In [98]:

train = pd.read_csv("../data/train_clean4.csv")
test = pd.read_csv("../data/test_clean2.csv")


In [99]:
train.columns

Index(['log_price', 'room_type', 'cleaning_fee', 'bed_type',
       'cancellation_policy', 'city', 'host_experience', 'beds',
       'host_response_rate', 'review_scores_rating', 'number_of_reviews',
       'accommodates', 'bathrooms', 'bedrooms', 'property_type',
       'mean_log_price', 'neighbourhood_cluster', 'neighbourhood'],
      dtype='object')

In [100]:
train.head()

,log_price,room_type,cleaning_fee,bed_type,cancellation_policy,city,host_experience,beds,host_response_rate,review_scores_rating,number_of_reviews,accommodates,bathrooms,bedrooms,property_type,mean_log_price,neighbourhood_cluster,neighbourhood
0,4.499810,Entire home/apt,1,Real Bed,flexible,NYC,poco_experimentado,1,80,96,9,2,1.0,1,Apartment,4.594133,5.0,Astoria
1,3.970292,Private room,1,Futon,strict,NYC,experimentado,1,94,100,8,2,1.0,1,Apartment,4.594133,4.0,Hamilton Heights
2,4.787492,Entire home/apt,1,Real Bed,moderate,NYC,muy_experimentado,1,100,100,12,4,1.0,1,Apartment,5.072061,4.0,Bedford-Stuyvesant
3,4.700480,Entire home/apt,0,Real Bed,moderate,NYC,muy_experimentado,1,94,90,2,3,1.1,1,Apartment,4.861277,6.0,Clinton Hill
4,5.192957,Entire home/apt,1,Real Bed,strict,NYC,muy_experimentado,1,94,80,1,3,1.0,1,Apartment,4.861277,6.0,Long Island City


# Selección de las columnas necesarias para el modelo

In [80]:
cols = ['log_price', 'room_type', 'city','bathrooms', 'bedrooms', 'mean_log_price', 'neighbourhood_cluster']

colst =['room_type', 'city','bathrooms', 'bedrooms', 'mean_log_price', 'neighbourhood_cluster']

train = train[cols]
test = test[colst]

In [81]:
train.head()

,log_price,room_type,city,bathrooms,bedrooms,mean_log_price,neighbourhood_cluster
0,4.499810,Entire home/apt,NYC,1.0,1,4.594133,5.0
1,3.970292,Private room,NYC,1.0,1,4.594133,4.0
2,4.787492,Entire home/apt,NYC,1.0,1,5.072061,4.0
3,4.700480,Entire home/apt,NYC,1.1,1,4.861277,6.0
4,5.192957,Entire home/apt,NYC,1.0,1,4.861277,6.0


# Empleo de AutoML con H2o

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.391-b13, mixed mode)
  Starting server from C:\Users\daarr\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\daarr\AppData\Local\Temp\tmpxlc0e674
  JVM stdout: C:\Users\daarr\AppData\Local\Temp\tmpxlc0e674\h2o_daarr_started_from_python.out
  JVM stderr: C:\Users\daarr\AppData\Local\Temp\tmpxlc0e674\h2o_daarr_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_daarr_aotaov
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.475 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [101]:
h2train = h2o.H2OFrame(train)

h2test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [102]:
y = 'log_price'

X = [c for c in h2train.columns if c != 'log_price']

In [113]:
# inicia auto-machine-learning


automl = H2OAutoML(max_models=20,        # nº de modelos
                   seed=42,              # random state
                   max_runtime_secs=600,
                   sort_metric='RMSE',stopping_metric = 'RMSE'
                  )

In [114]:
automl.train(x=X, y=y, training_frame=h2train)

AutoML progress: |
01:17:00.655: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_10_20231128_11700


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    63                 63                          271114                 7            7            7             64            126           107.81

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.11462356180918247
RMSE: 0.338561016375457
MAE: 0.24659471472672082
RMSLE: 0.05826361206855208
Mean Residual Deviance: 0.11462356180918247

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.15475867258715104
RMSE: 0.3933937881908547
MAE: 0.28435622695381585
RMSLE: 0.06739493122237486
Mean Residual Deviance: 0.15475867258715104

Cross-Validation Metrics Summary: 
                        mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     0.284356   0.00165411  0.281991      0.284799      0.285353      0.286185      0.283454
mean_residual_deviance  0.154759   0.00399182  0.151403      0.152946      0.161619      0.154503      0.153323
mse                     0.154759   0.00399182  0.151403      0.152946      0.161619      0.154503      0.153323
r2                      0.700843   0.0082394   0.705842      0.702629      0.686747      0.701502      0.707494
residual_deviance       0.154759   0.00399182  0.151403      0.152946      0.161619      0.154503      0.153323
rmse                    0.393368   0.00503905  0.389105      0.391083      0.402019      0.393068      0.391565
rmsle                   0.0673876  0.00111281  0.0664409     0.0667908     0.0691769     0.067749      0.0667804

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-11-28 01:21:12  9.326 sec   0                  0.719305         0.562737        0.517399
    2023-11-28 01:21:12  9.479 sec   5                  0.545024         0.416326        0.297051
    2023-11-28 01:21:12  9.625 sec   10                 0.459462         0.344425        0.211105
    2023-11-28 01:21:12  9.780 sec   15                 0.414009         0.306627        0.171403
    2023-11-28 01:21:12  9.944 sec   20                 0.388502         0.285601        0.150934
    2023-11-28 01:21:12  10.104 sec  25                 0.374083         0.273919        0.139938
    2023-11-28 01:21:13  10.358 sec  30                 0.363231         0.265694        0.131937
    2023-11-28 01:21:13  10.509 sec  35                 0.356627         0.260503        0.127183
    2023-11-28 01:21:13  10.685 sec  40                 0.351538         0.256604        0.123579
    2023-11-28 01:21:13  10.834 sec  45                 0.348658         0.254305        0.121563
    2023-11-28 01:21:13  11.003 sec  50                 0.344992         0.251491        0.119019
    2023-11-28 01:21:13  11.156 sec  55                 0.342643         0.249752        0.117404
    2023-11-28 01:21:14  11.352 sec  60                 0.339514         0.247393        0.11527
    2023-11-28 01:21:14  11.451 sec  63                 0.338561         0.246595        0.114624

Variable Importances: 
variable               relative_importance    scaled_importance    percentage
---------------------  ---------------------  -------------------  ------------
room_type              38533.8                1                    0.334702
bedrooms               18196.5                0.472222     

In [115]:
lider = automl.leaderboard

lider.head(15)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_2_AutoML_10_20231128_11700,0.393394,0.154759,0.284356,0.0673949,0.154759
GBM_5_AutoML_10_20231128_11700,0.394333,0.155499,0.28531,0.0675679,0.155499
GBM_1_AutoML_10_20231128_11700,0.39514,0.156136,0.285817,0.0676662,0.156136
GBM_4_AutoML_10_20231128_11700,0.396221,0.156991,0.285999,0.0677953,0.156991
GBM_3_AutoML_10_20231128_11700,0.396277,0.157035,0.286052,0.0678261,0.157035
GBM_grid_1_AutoML_10_20231128_11700_model_1,0.398641,0.158915,0.288546,0.0683129,0.158915
XRT_1_AutoML_10_20231128_11700,0.400691,0.160553,0.291573,0.0684166,0.160553
DRF_1_AutoML_10_20231128_11700,0.403195,0.162566,0.289193,0.0687339,0.162566
DeepLearning_1_AutoML_10_20231128_11700,0.404376,0.16352,0.293054,0.0690237,0.16352
GLM_1_AutoML_10_20231128_11700,0.416367,0.173362,0.302981,0.0712919,0.173362


In [116]:
y_pred = automl.leader.predict(h2test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\daarr\anaconda3\Lib\site-packages\h2o\job.py:81: UserWarning: Test/Validation dataset column 'neighbourhood' has levels not trained on: ["Chevy Chase, MD", "Government Center", "Graniteville", "New Dorp Beach", "Observatory Circle", "Suitland-Silver Hill, MD"]
  warnings.warn(w)


In [117]:
y_pred = y_pred.as_data_frame()

# Generación del documento de entrega

In [118]:
entrega3 = pd.read_csv("../data/muestra.csv")

In [119]:
entrega3["log_price"] = y_pred

In [120]:
entrega3.head(10)

,id,log_price
0,0,5.413162
1,1,5.211140
2,2,3.624119
3,3,5.234081
4,4,4.640233
5,5,5.307480
6,6,5.919884
7,7,4.147835
8,8,4.920580
9,9,5.150584


In [111]:
entrega3.to_csv("../data/entrega12.csv", index=False)

# Aplicación de otros modelos previamente testeados

In [ ]:


cols = ['property_type','log_price', 'room_type',
       'accommodates', 'bathrooms', 'cancellation_policy',
        'city','bedrooms', 'host_experience', 'neighbourhood_cluster', 'jacuzzi', 'fireplace', 
        'luxury', 'penthouse']

bnb2 = bnb2[cols]
bnb3 = bnb3[cols]

# One Hot Encoding

In [ ]:

bnb2 = pd.get_dummies(bnb2, columns=['cancellation_policy','room_type','city','host_experience','property_type'], drop_first=True)

bnb3 = pd.get_dummies(bnb3, columns=['cancellation_policy','room_type','city','host_experience','property_type'], drop_first=True)


# Separación del Dataframe en Train y Test

In [ ]:
X_train = bnb2.drop('log_price', axis=1)

y_train = bnb2.log_price

X_test = bnb3.drop('log_price', axis=1)

y_test= bnb3.log_price

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Normalización

In [ ]:
escalador = StandardScaler()    # inicia el normalizador
 
escalador.fit(X_train[['accommodates', 'bathrooms','bedrooms']])   # ajusta segun train, saca media y std
 
X_train[['accommodates', 'bathrooms','bedrooms']] = escalador.fit_transform(X_train[['accommodates', 'bathrooms','bedrooms']])   # transforma train

In [ ]:
X_test[['accommodates', 'bathrooms','bedrooms']] = escalador.transform(X_test[['accommodates', 'bathrooms','bedrooms']])  # transforma test

In [ ]:
X_train.head()

In [ ]:
X_test.head()

# Entrenamiento de los modelos

In [ ]:
cbr = CBR( depth = 8, iterations= 500, learning_rate= 0.05)    # inicia el modelo


cbr.fit(X_train, y_train)   # entrena el modelo

In [ ]:
lgbmr = LGBMR(colsample_bytree=0.8, learning_rate=0.1, max_depth=5, n_estimators=200, subsample=0.8,force_row_wise=True, n_jobs=8)
lgbmr.fit(X_train, y_train) 

In [ ]:
# rfr = RFR(max_depth= 10, n_estimators= 150)

# rfr.fit(X_train, y_train)

In [ ]:
# xgbr = XGBR(
#     objective='reg:squarederror',  # Puedes usar 'reg:squaredlogerror' para regresión logarítmica
#     max_depth=5,
#     learning_rate=0.1,
#     n_estimators=200
# )

# xgbr.fit(X_train, y_train)

In [ ]:
# mlp = MLPRegressor(activation= 'tanh', alpha= 0.0001, hidden_layer_sizes= (50,))

# mlp.fit(X_train, y_train)

In [ ]:
# gbr = GBR(learning_rate= 0.1, max_depth= 5, n_estimators= 150)    # inicia el modelo


# gbr.fit(X_train, y_train)   # entrena el modelo

# Predicción del modelo

In [ ]:
# y_pred1 = rfr.predict(X_test)
# y_pred2 = xgbr.predict(X_test)
# y_pred3 = mlp.predict(X_test)
# y_pred4 = gbr.predict(X_test)
y_pred5 = cbr.predict(X_test)
y_pred6 = lgbmr.predict(X_test)

# Generación del documento de entrega

In [ ]:
entrega3 = pd.read_csv("../data/muestra.csv")

In [69]:
entrega3["log_price"] = y_pred6


NameError: name 'y_pred6' is not defined

In [ ]:
entrega3.head(10)

In [ ]:
entrega3.to_csv("../data/entrega7.csv", index=False)